In [2]:
import pandas as pd
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np 
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from functools import partial
from src.bootstrap import bootstrap_eval

def get_in_ref(peptide, hla, ref):
    return len(ref.query('Peptide==@peptide and HLA == @hla'))==1

In [3]:
icore = pd.read_csv('../output/PRIME_preds_cedar_onehot_None_icore_mut_only_rank.csv')
pep = pd.read_csv('../output/PRIME_preds_cedar_onehot_None_icore_mut_only_rank.csv')
nepdb = pd.read_csv('../output/NEPDB_preds_cedar_onehot_None_icore_mut_only_rank.csv')

icore['in_nepdb'] = icore.apply(lambda x: get_in_ref(x['Peptide'], x['HLA'], ref=nepdb), axis=1)
pep['in_nepdb'] = pep.apply(lambda x: get_in_ref(x['Peptide'], x['HLA'], ref=nepdb), axis=1)

icore = icore.query('not in_nepdb')
pep = pep.query('not in_nepdb')

In [9]:
p = 'pred' if 'pred' in icore.columns else 'mean_pred'

boot_icore = bootstrap_eval(icore[p].values, icore['agg_label'].values, n_rounds=10000, n_jobs=8, auc01=True)
print(len(boot_icore))
boot_icore = boot_icore.assign(input_type='icore_mut', evalset='PRIME')

Sampling


Bootstrapping rounds:   0%|          | 0/10000 [00:00<?, ?it/s]

Making results DF and curves
10000
Sampling


Bootstrapping rounds:   0%|          | 0/10000 [00:00<?, ?it/s]

Making results DF and curves


In [15]:
boot_pep = bootstrap_eval(pep[p].values, pep['agg_label'].values, n_rounds=10000, n_jobs=8, auc01=True).assign(input_type='Peptide', evalset='PRIME')

Sampling


Bootstrapping rounds:   0%|          | 0/10000 [00:00<?, ?it/s]

Making results DF and curves


In [17]:

boot = pd.concat([boot_icore, boot_pep], axis=0).assign(weight='None', key='only_rank')
boot.to_csv('./PRIME_filtered_baseline.csv')

In [18]:
boot

,auc,auc_01,precision,accuracy,AP,id,input_type,evalset,weight,key
0,0.679144,0.558129,0.024479,0.552902,0.111860,0_16532143381525,icore_mut,PRIME,None,only_rank
0,0.591759,0.513602,0.019626,0.562371,0.020797,1_1061590028695,icore_mut,PRIME,None,only_rank
0,0.616249,0.523740,0.024479,0.553726,0.027490,2_16082104450083,icore_mut,PRIME,None,only_rank
0,0.606370,0.514651,0.016409,0.576369,0.053064,3_1898856662319,icore_mut,PRIME,None,only_rank
0,0.665110,0.522967,0.017824,0.565253,0.021665,4_1146994453171,icore_mut,PRIME,None,only_rank
...,...,...,...,...,...,...,...,...,...,...
0,0.655246,0.536961,0.024668,0.573899,0.058998,9995_22751068879253,Peptide,PRIME,None,only_rank
0,0.673729,0.558084,0.018147,0.573487,0.034999,9996_7172282207344,Peptide,PRIME,None,only_rank
0,0.616154,0.540668,0.016473,0.575957,0.056995,9997_12831301130163,Peptide,PRIME,None,only_rank
0,0.675094,0.560425,0.026490,0.571429,0.066224,9998_12411499925411,Peptide,PRIME,None,only_rank
